### Igraph를 통한 네트워크 분석
: 대규모 데이터 처리 가능 / 그룹화 자동 수행

-> pyvis & plotly를 통한 시각화

### Import Libraries & Google Drive 마운트 & 파일 읽기

In [ ]:
!pip install pandas python-igraph cairocffi pyvis transformers emoji==0.6.0



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.6 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49719 sha256=12d2cbec613b9978cd17bafef98c2afaf407ce375040c68babc58e020a1ccb66
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji


In [ ]:
import pandas as pd
from igraph import Graph, plot
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from collections import Counter
from scipy.spatial import ConvexHull
import numpy as np
from matplotlib.cm import get_cmap
from matplotlib.patches import Polygon
from pyvis.network import Network
from IPython.core.display import display, HTML
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 파일 경로 설정
post = '/content/drive/MyDrive/BADA 산학/dataset/posts_final_1119_cleaned.csv'

# CSV 파일 읽기
df_post = pd.read_csv(post)
columns=['Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0']
df_post=df_post.drop(labels=columns,axis=1)

In [ ]:
df_post

,post_id,username,datetime,description,hashtags,music,video_url,thumbnail_url,video_length,filter_effect,location,blue_badge,collected_time,query_id,follower_count,user_group,like_count,comment_count,save_count
0,7211559032480468266,myfahlo,2023-3-18,These @myfahlo animal tracking bracelets are a...,#seaturtle #giraffe #lions #animallover,Promoted Music,https://www.tiktok.com/@myfahlo/video/72115590...,NaN,NaN,NaN,NaN,0,2024-11-11 05:14:47,0,138100.0,2,407600,4436,14600.0
1,7221676409432001838,7thavenueco,2023-4-14,Say hello to the deepest modular sofa on the m...,NaN,Promoted Music,https://www.tiktok.com/@7thavenueco/video/7221...,NaN,NaN,NaN,NaN,1,2024-11-11 05:14:46,0,24900.0,1,39800,745,5606.0
2,7230070392915021102,madisonacash,2023-5-6,Replying to,NaN,Promoted Music,https://www.tiktok.com/@madisonacash/video/723...,NaN,NaN,NaN,NaN,0,2024-11-11 01:35:55,0,58800.0,1,66900,649,5232.0
3,7267281019781369131,7thavenueco,2023-8-15,What’s the craziest thing you’ve spilled on yo...,#sectionalsofa #sofa #homedecor #interiordesign,Promoted Music,https://www.tiktok.com/@7thavenueco/video/7267...,NaN,NaN,NaN,NaN,1,2024-11-11 05:15:14,0,24900.0,1,57400,892,5220.0
4,7336487071323786539,icm_triplets,2-17,dc:,#cleared #dance #dancechallenge #pourtoi #fyp ...,original sound - 𝐩𝐡𝐫𝐞𝐥𝐥.,https://www.tiktok.com/@icm_triplets/video/733...,NaN,NaN,NaN,NaN,0,2024-11-11 05:15:08,0,2400000.0,3,13800000,56400,1700000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,7434025114783485200,lovec.com8,11h ago,Made in Bugis.,['y p s i n d r a p'],suara asli - dian,https://www.tiktok.com/@lovec.com8/video/74340...,NaN,NaN,NaN,NaN,0,2024-11-07 01:42:19,0,19700.0,1,703,12,19.0
2003,7434129756435565832,lu_xiii,4h ago,It's a waste of life to be in such a mess.,"['trendy,making up,tips for makeup,making up r...",nhạc nền - Lu,https://www.tiktok.com/@lu_xiii/video/74341297...,NaN,NaN,NaN,NaN,0,2024-11-07 01:42:11,0,246300.0,2,71100,689,3741.0
2004,7434130536936901906,adellacos,3h ago,Is this acceptable?,"['for you', 'fyp']",original sound - Deng,https://www.tiktok.com/@adellacos/video/743413...,NaN,NaN,NaN,NaN,0,2024-11-07 00:11:44,0,3700000.0,3,175800,733,9340.0
2005,7434171420764114194,wuncole,1h ago,"I just want to give her a hug, I am truly shoc...","['kamala,harris,donald,trump,u-election']",original sound - wuncole,https://www.tiktok.com/@wuncole/video/74341714...,NaN,NaN,NaN,NaN,0,2024-11-07 01:42:04,0,32700.0,1,86600,2250,16600.0


#### 사용한 데이터셋
1) post(241007,241010,241011)

: post_id, username, description, hashtags, blue_badge

2) post count(241007,241010,241011)

: post_id, like_count, save_count

=> post_id 기준으로 재크롤링


3) user count(241007,241010,241011)

: username, follower_count

=> username 기준으로 재크롤링


#### 네트워크 구조

노드: post_id

엣지: hashtags, description

### Hashtag 엣지 생성

#### Hashtag 전처리

In [ ]:
import re

# 해시태그 문자열을 리스트로 변환하는 함수
def split_hashtags(hashtag_str):
    if isinstance(hashtag_str, str):
        # '#' 기호로 시작하는 경우
        if '#' in hashtag_str:
            return [tag.strip() for tag in hashtag_str.split('#') if tag.strip()]
        # 리스트 형태의 문자열인 경우
        elif '[' in hashtag_str:
            # 문자열에서 '[', ']', ' 제거하고 쉼표로 분리
            return [tag.strip().strip("'").strip() for tag in hashtag_str.strip('[]').split(',') if tag.strip()]
    return []

# 해시태그 분리 적용
df_post['hashtag_list'] = df_post['hashtags'].apply(split_hashtags)

def clean_hashtags(hashtag_list):
    """
    해시태그 데이터를 처리:
    1. 모든 해시태그를 소문자로 변환
    2. 해시태그에서 이모지 제거
    3. 중복 해시태그 제거
    """
    if not isinstance(hashtag_list, list):
        return []

    # 이모지를 제거하는 정규 표현식
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese characters
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010FFFF"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200D"
        u"\u23CF"
        u"\u23E9"
        u"\u231A"
        u"\uFE0F"  # dingbats
        u"\u3030"
        "]+",
        flags=re.UNICODE
    )

    # 해시태그를 처리
    cleaned_hashtags = []
    for hashtag in hashtag_list:
        # 소문자로 변환 및 이모지 제거
        cleaned = emoji_pattern.sub("", hashtag.lower())
        cleaned_hashtags.append(cleaned)

    # 중복 제거
    return list(set(cleaned_hashtags))

# 해시태그 데이터 처리
df_post['cleaned_hashtags'] = df_post['hashtag_list'].apply(clean_hashtags)

# 삭제 기준이 되는 단어 리스트
remove_keywords = ['fyp', 'foryou', 'viral','fyy','tiktok','trending','trend', 'funny','pov','xuhong','parati','meme','foru','haha', 'for you page','for you',
                   'foryoupage', 'tiktokfamous', 'viralpost', 'video', 'justforfun', 'follow']

# 필터링 함수
def remove_hashtags(hashtags, keywords):
    return [tag for tag in hashtags if not any(keyword in tag.lower() for keyword in keywords)]

# 결측값이 아닌 경우에만 처리
df_post['filtered_hashtags'] = df_post['cleaned_hashtags'].apply(
    lambda tags: [tag for tag in tags if not any(keyword in tag for keyword in remove_keywords)] if tags else []
)

# 결과 확인
df_post[['post_id','hashtag_list', 'cleaned_hashtags', 'filtered_hashtags']]

,post_id,hashtag_list,cleaned_hashtags,filtered_hashtags
0,7211559032480468266,"[seaturtle, giraffe, lions, animallover]","[lions, animallover, seaturtle, giraffe]","[lions, animallover, seaturtle, giraffe]"
1,7221676409432001838,[],[],[]
2,7230070392915021102,[],[],[]
3,7267281019781369131,"[sectionalsofa, sofa, homedecor, interiordesign]","[sectionalsofa, sofa, interiordesign, homedecor]","[sectionalsofa, sofa, interiordesign, homedecor]"
4,7336487071323786539,"[cleared, dance, dancechallenge, pourtoi, fyp,...","[triplets, pourtoi, dance, fyp, slide, dancech...","[triplets, pourtoi, dance, slide, dancechallen..."
...,...,...,...,...
2002,7434025114783485200,[y p s i n d r a p],[y p s i n d r a p],[y p s i n d r a p]
2003,7434129756435565832,"[trendy, making up, tips for makeup, making up...","[making up review, trendy, tips for makeup, ma...","[making up review, tips for makeup, making up]"
2004,7434130536936901906,"[for you, fyp]","[fyp, for you]",[]
2005,7434171420764114194,"[kamala, harris, donald, trump, u-election]","[u-election, trump, harris, kamala, donald]","[u-election, trump, harris, kamala, donald]"


#### Hashtag 엣지 생성

In [ ]:
# 해시태그 기반 엣지 생성
post_hashtags = df_post[['post_id', 'filtered_hashtags']].dropna(subset=['filtered_hashtags'])
hashtag_to_posts = defaultdict(list)
post_to_hashtags = defaultdict(set)

# 게시물과 해시태그 매핑
for _, row in post_hashtags.iterrows():
    post_id = row['post_id']
    hashtags = row['filtered_hashtags']
    post_to_hashtags[post_id] = set(hashtags)
    for tag in hashtags:
        hashtag_to_posts[tag].append(post_id)

# 해시태그 엣지 생성
edges_hashtag_limited = set()  # 최종 엣지를 저장할 집합

for posts in hashtag_to_posts.values():
    for i in range(len(posts)):
        for j in range(i + 1, len(posts)):
            # 두 게시물의 해시태그 교집합 계산
            common_hashtags = post_to_hashtags[posts[i]] & post_to_hashtags[posts[j]]
            if len(common_hashtags) >= 3:  # 교집합이 3개 이상인 경우에만 엣지 추가
                edges_hashtag_limited.add((posts[i], posts[j]))

# 엣지를 리스트로 변환
edges_hashtag = list(edges_hashtag_limited)

### Description 엣지 생성

In [ ]:
# 디스크립션 기반 엣지 생성
post_ids = df_post['post_id'].astype(str).tolist()
descriptions = df_post['description'].tolist()
filtered_hashtags = df_post['filtered_hashtags'].tolist()

# 모델 로드
tokenizer = AutoTokenizer.from_pretrained("Twitter/twhin-bert-base")
model = AutoModel.from_pretrained("Twitter/twhin-bert-base")
embeddings = []

# 임베딩 생성
for i, desc in enumerate(descriptions):
    if pd.notna(desc) and desc.strip():
        inputs = tokenizer(desc, return_tensors="pt", truncation=True, padding=True, max_length=128)
        outputs = model(**inputs)
        embeddings.append(outputs.pooler_output.cpu().detach().numpy().flatten())
    else:
        hashtags = filtered_hashtags[i]
        if hashtags:
            hashtags_str = " ".join(hashtags)
            inputs = tokenizer(hashtags_str, return_tensors="pt", truncation=True, padding=True, max_length=128)
            outputs = model(**inputs)
            embeddings.append(outputs.pooler_output.cpu().detach().numpy().flatten())
        else:
            embeddings.append(None)

# 유효한 임베딩과 유사도 계산
valid_embeddings = np.array([emb for emb in embeddings if emb is not None])
similarities = cosine_similarity(valid_embeddings)

# 유효한 포스트 ID 매핑
valid_indices = [i for i, emb in enumerate(embeddings) if emb is not None]
valid_post_ids = [post_ids[idx] for idx in valid_indices]

# 엣지 생성
node_edge_count_description = defaultdict(int)
edges_description_limited = []

for i in range(len(similarities)):
    potential_edges = []

    for j in range(i + 1, len(similarities)):
        if similarities[i, j] > 0.95:  # 유사도 0.95 기준
            potential_edges.append((similarities[i, j], valid_post_ids[i], valid_post_ids[j]))

    # 유사도 기준으로 정렬
    potential_edges.sort(reverse=True, key=lambda x: x[0])

    # 최대 3개 엣지 제한 적용
    for _, post1, post2 in potential_edges:
        if node_edge_count_description[post1] < 3 and node_edge_count_description[post2] < 3:
            edges_description_limited.append((post1, post2))
            node_edge_count_description[post1] += 1
            node_edge_count_description[post2] += 1

edges_description = edges_description_limited

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


### 네트워크 생성 및 커뮤니티 탐지

In [ ]:
# 네트워크 생성 및 커뮤니티 탐지
g = Graph()
g.add_vertices(post_ids)

g.add_edges(edges_description)
g.add_edges([(str(e[0]), str(e[1])) for e in edges_hashtag])

g.vs['like_count'] = df_post['like_count'].fillna(0).astype(int).tolist()
g.vs['save_count'] = df_post['save_count'].fillna(0).astype(int).tolist()

random.seed(42)
np.random.seed(42)
communities = g.community_multilevel()
g.vs['community'] = communities.membership

community_ids = set(g.vs['community'])
community_sizes = {cid: len([v for v in g.vs if v['community'] == cid]) for cid in community_ids}
filtered_communities = {cid: size for cid, size in community_sizes.items() if size >= 70}
sorted_communities = sorted(filtered_communities.items(), key=lambda x: x[1], reverse=True)

valid_community_ids = set(filtered_communities.keys())
g.vs['community'] = [cid if cid in valid_community_ids else -1 for cid in g.vs['community']]

filtered_count = len(filtered_communities)

print(f"Detected {filtered_count} Communities")
print("Sorted Communities by Size (Filtered):")
for cid, size in sorted_communities:
    print(f"Community {cid}: {size} nodes")

Detected 9 Communities
Sorted Communities by Size (Filtered):
Community 53: 87 nodes
Community 21: 81 nodes
Community 10: 80 nodes
Community 14: 76 nodes
Community 17: 76 nodes
Community 2: 73 nodes
Community 7: 72 nodes
Community 27: 72 nodes
Community 23: 71 nodes


### 커뮤니티 별 핵심 주제 출력
멀티헤드 트랜스포머 활용해 커뮤니티별 핵심 디스크립션, 해시태그 추출
- transformer block = 3, multi-head = 8
- 커뮤니티 별 대표 벡터와의 유사도 상위 5개 노드에 대해 내용 출력
- 1) viral지표 2)  description 3) hashtag

In [ ]:
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch
import pandas as pd
from igraph import Graph

# Transformer 블록 정의 (Cross Attention 포함)
class TransformerBlock(torch.nn.Module):
    def __init__(self, input_dim, num_heads=8):
        super(TransformerBlock, self).__init__()
        self.multihead_attn = torch.nn.MultiheadAttention(embed_dim=input_dim, num_heads=num_heads, batch_first=True)
        self.ffn = torch.nn.Sequential(
            torch.nn.Linear(input_dim, input_dim * 4),
            torch.nn.ReLU(),
            torch.nn.Linear(input_dim * 4, input_dim)
        )

    def forward(self, query, key, value):
        attn_output, _ = self.multihead_attn(query, key, value)  # Cross Attention
        query = query + attn_output  # Residual Connection
        ffn_output = self.ffn(query)
        query = query + ffn_output  # Residual Connection
        return query

# Apply Transformer 블록 여러 번
def apply_transformer_blocks(query_vectors, key_value_vectors, num_blocks=1, input_dim=64):
    query_vectors = torch.tensor(query_vectors, dtype=torch.float32).unsqueeze(0)  # (Batch, Sequence, Features)
    key_value_vectors = torch.tensor(key_value_vectors, dtype=torch.float32).unsqueeze(0)  # (Batch, Sequence, Features)

    transformer_blocks = [TransformerBlock(input_dim) for _ in range(num_blocks)]
    for block in transformer_blocks:
        query_vectors = block(query_vectors, key_value_vectors, key_value_vectors)  # Cross Attention
    return query_vectors.squeeze(0).detach().numpy()  # Remove batch dimension and return as numpy array

# Closest descriptions, hashtags 반환 함수
def get_closest_descriptions(rep_vector, community_vectors, descriptions, hashtags, top_k=3):
    similarities = cosine_similarity([rep_vector], community_vectors)[0]
    top_indices = np.argsort(similarities)[::-1][:top_k]
    return [(descriptions[idx], hashtags[idx]) for idx in top_indices]

# t-SNE로 768차원을 64차원으로 축소
tsne = TSNE(n_components=64, random_state=42, method='exact')
reduced_embeddings = tsne.fit_transform(valid_embeddings)

# 커뮤니티 분석 결과 저장 리스트
community_analysis = []

# 커뮤니티 별로 Cross Attention 및 Transformer 블록 적용
for cid, size in sorted_communities:
    # g.vs 인덱스를 valid_indices와 매핑하여 유효한 인덱스만 사용
    node_indices = [
        valid_indices.index(idx) for idx, v in enumerate(g.vs) if v['community'] == cid and idx in valid_indices
    ]

    if not node_indices:  # 해당 커뮤니티에 유효한 노드가 없으면 건너뛰기
        continue

    community_vectors = reduced_embeddings[node_indices]
    community_descriptions = [df_post.iloc[valid_indices[idx]]['description'] for idx in node_indices]
    community_hashtags = [df_post.iloc[valid_indices[idx]]['filtered_hashtags'] for idx in node_indices]
    community_likes = [df_post.iloc[valid_indices[idx]]['like_count'] for idx in node_indices]

    # 바이럴 지표 계산
    total_likes = sum(community_likes)

    # Cross Attention 및 Transformer 블록 반복
    updated_vectors = apply_transformer_blocks(
        query_vectors=community_vectors,   # Query: 현재 커뮤니티의 벡터
        key_value_vectors=community_vectors,  # Key/Value: 같은 커뮤니티의 벡터
        num_blocks=3,
        input_dim=64
    )

    # 대표 벡터 생성
    representative_vector = np.mean(updated_vectors, axis=0)

    # 대표 벡터와 가장 가까운 디스크립션 5개 및 필터된 해시태그 출력
    closest_descriptions = get_closest_descriptions(
        representative_vector, updated_vectors, community_descriptions, community_hashtags, top_k=10
    )

    # 분석 결과를 커뮤니티 분석 리스트에 추가
    community_data = {
        "Community": cid,
        "Size": size,
        "Total Likes": total_likes,
        "Descriptions and Hashtags": [{"description": desc, "hashtags": tags} for desc, tags in closest_descriptions]
    }
    community_analysis.append(community_data)

### 시각화 저장 및 결과 데이터 프레임 출력
- Plotly 사용한 인터랙티브 시각화
- 블루뱃지 유무에 따라 투명도 차이
- 결과 HTML로 저장

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import random


def summarize_and_visualize_communities(
    community_analysis_results,
    graph,
    df_post,
    output_filename="tiktok_posts_visualization.html"
):
    """
    Parameters:
        community_analysis_results (list of dict): 커뮤니티 분석 결과.
        graph (igraph.Graph): TikTok 데이터의 그래프 객체.
        df_post (pd.DataFrame): 노드 속성을 포함한 데이터프레임 (blue_badge 추가에 필요).
        output_filename (str): HTML로 저장될 시각화 파일 이름.

    Returns:
        pd.DataFrame: 커뮤니티 분석 결과 데이터프레임.
        None: 시각화는 HTML로 저장.
    """

    # 노드별 시각화 차이를 위해 blue_badge 속성 추가
    if "blue_badge" not in graph.vs.attributes():
        blue_badge_mapping = dict(zip(df_post["post_id"].astype(str), df_post["blue_badge"]))
        graph.vs["blue_badge"] = [blue_badge_mapping.get(v["name"], 0) for v in graph.vs]  # 기본값은 0

    # 커뮤니티 분석 결과를 데이터프레임으로 변환
    df_analysis = pd.DataFrame(community_analysis_results)

    # 유효한 커뮤니티 노드만 필터링
    valid_nodes = [v.index for v in graph.vs if v["community"] != -1]
    subgraph = graph.subgraph(valid_nodes)  # 유효한 노드로 서브그래프 생성

    # 고정된 커뮤니티 색상 설정
    predefined_colors = ["orange", "green", "blue","yellow", "red", "purple"]
    community_ids = list(set(subgraph.vs["community"]))
    community_colors = {
        cid: predefined_colors[i % len(predefined_colors)]
        for i, cid in enumerate(community_ids)
    }

    # 기본 레이아웃 설정
    layout = subgraph.layout("fr")
    x_coords = [layout[i][0] * 10 for i in range(len(subgraph.vs))]
    y_coords = [layout[i][1] * 10 for i in range(len(subgraph.vs))]

    # Plotly Figure 생성
    fig = go.Figure()

    # 엣지 추가
    edge_x = []
    edge_y = []
    for edge in subgraph.es:
        source = edge.source
        target = edge.target
        edge_x.extend([x_coords[source], x_coords[target], None])
        edge_y.extend([y_coords[source], y_coords[target], None])

    fig.add_trace(
        go.Scatter(
            x=edge_x,
            y=edge_y,
            mode="lines",
            line=dict(width=0.5, color="lightgrey"),
            hoverinfo="none",  # 엣지의 Hover 정보 제거
        )
    )

    # 노드 추가
    node_x = []
    node_y = []
    node_color = []
    node_text = []
    node_size = []  # 노드 크기를 저장할 리스트 추가
    for i, vertex in enumerate(subgraph.vs):
        node_x.append(x_coords[i])
        node_y.append(y_coords[i])
        node_color.append(community_colors[vertex["community"]])  # 고정 색상 매핑
        node_text.append(
            f"Community: {vertex['community']}<br>"
            f"Post ID: {vertex['name']}<br>"
            f"Likes: {vertex['like_count']}<br>"
            f"Saves: {vertex['save_count']}"
        )
        # 블루뱃지 여부에 따라 크기 설정
        if vertex["blue_badge"] == 1:
            node_size.append(25)  # 블루뱃지가 있는 노드
        else:
            node_size.append(15)  # 블루뱃지가 없는 노드

    fig.add_trace(
        go.Scatter(
            x=node_x,
            y=node_y,
            mode="markers",
            marker=dict(
                size=node_size,  # 노드 크기 설정
                color=node_color,
                opacity=[
                    1 if vertex["blue_badge"] == 1 else 0.4 for vertex in subgraph.vs
                ],
                line=dict(width=1, color="black"),
            ),
            text=node_text,
            hoverinfo="text",  # 노드 정보 Hover 활성화
        )
    )

    # 레이아웃 설정
    fig.update_layout(
        title="TIKTOK Posts Visualization",
        showlegend=False,
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        height=600,
        width=1200,
    )

    # 시각화 HTML로 저장
    fig.write_html(output_filename)
    print(f"시각화 결과가 '{output_filename}'에 저장되었습니다.")

    # 데이터프레임 반환
    return df_analysis


# 최종 결과를 데이터프레임으로 반환
final_result = summarize_and_visualize_communities(community_analysis, g, df_post)

# 결과 출력
final_result


시각화 결과가 'tiktok_posts_visualization.html'에 저장되었습니다.


,Community,Size,Total Likes,Descriptions and Hashtags
0,53,87,26688872,"[{'description': nan, 'hashtags': ['anime', 'a..."
1,21,81,90470175,"[{'description': 'late for trend sorry', 'hash..."
2,10,80,69103572,"[{'description': 'Lol what a mess.🤦🏻‍♂️', 'has..."
3,14,76,55770508,"[{'description': nan, 'hashtags': ['chrollo', ..."
4,17,76,52286796,"[{'description': 'Sorry for my accent 😔', 'has..."
5,2,73,62445456,"[{'description': ':/ happy saturday', 'hashtag..."
6,7,72,25118397,"[{'description': 'I love you so much, our line..."
7,27,72,40827447,[{'description': 'There is no clear text to tr...
8,23,71,46152607,"[{'description': 'Hola b*tches.', 'hashtags': ..."
